In [9]:
import time
import torch
import random
from torch import nn
import numpy as np
from matplotlib import pyplot as plt
import nbimporter
from OptiEngine import FEM_2D, Config, Timer, optimize

In [2]:
from NeuralNetwork_First_Iter import TopologyGenerator_FirstIter
from NeuralNetwork_Predict_First_Iter import FirstIter_Predictor

In [3]:
predictor = FirstIter_Predictor()
generator = TopologyGenerator_FirstIter()

In [4]:
fiter_predictor_loadname = 'NeuralNetwork_Predict_First_Iter_params'
topology_generator_loadname = 'NeuralNetwork_FirstIter_params'

In [5]:
predictor.load_state_dict(torch.load(fiter_predictor_loadname))
generator.load_state_dict(torch.load(topology_generator_loadname))

<All keys matched successfully>

In [6]:
def gen_design(task, predictor, generator):
    fiter_predict = predictor(torch.FloatTensor(task).reshape(1,1,30,30)).detach().reshape(29,29)
    topology_predict = generator(torch.FloatTensor(fiter_predict).reshape(1,1,29,29)).detach().reshape(29,29)
    return topology_predict

In [7]:
def load_data(path, name_pattern, n_samples):
    pattern = f'{path}\\{name_pattern}_'
    fiter_pattern = f'{path}\\{name_pattern}_first_iter_'
    
    # Loading first sample
    tasks = np.loadtxt(f'{pattern}{0}_task.txt')
    f_iters = np.loadtxt(f'{fiter_pattern}{0}_result.txt')
    
    for sample_i in range(1, n_samples):
        new_task = np.loadtxt(f'{pattern}{sample_i}_task.txt')
        new_fiter = np.loadtxt(f'{fiter_pattern}{sample_i}_result.txt')
        tasks = np.concatenate([tasks, new_task], axis = 0)
        f_iters = np.concatenate([f_iters, new_fiter], axis = 0)
        
    return (tasks.reshape(n_samples, 30, 30), f_iters.reshape(n_samples, 29, 29))

In [11]:
overall_simp_time = 0
overall_nn_time = 0
timer = Timer()

config = Config()

nx = 30
ny = 30

config.nx = nx
config.ny = ny

config.vol_frac = 0.3
config.penalize = 3

config.dense_delta = 0.01
config.max_iter = 40

config.E0 = 1
config.E_void = 1e-9

config.filter_r = 1.5
config.move = 0.2

config.fem_save_iters = []

for i in range(n_samples):
    # Solving with SIMP
    fix_1_y = random.randint(0, 29)
    fix_2_y = fix_1_y
    while (fix_2_y == fix_1_y):
        fix_2_y = random.randint(0, 29)
        
    load_y = random.randint(0,29)
    load_x = random.randint(0,15)
    
    FEM_solver = FEM_2D(config.nx, config.ny, config.penalize)
    FEM_solver.fix_node(fix_1_y, 29, 1, 1)
    FEM_solver.fix_node(fix_2_y, 29, 1, 1)
    FEM_solver.apply_load(load_y, load_x, 0, -1)
    
    timer.start()
    FEM_solver.form_elements()
    config.H, config.sH = FEM_solver.prepare_filter()
    xphys_opt = optimize(FEM_solver, config, False, False)
    overall_simp_time += timer.time_passed()
    
    # Solving with NN
    task = np.zeros((30, 30))
    task[fix_1_y, 29] = 1
    task[fix_2_y, 29] = 1
    task[load_y, load_x] = -1
    
    timer.start()
    design = gen_design(task, predictor, generator)
    overall_nn_time += timer.time_passed()
    
    if ((i+1) % 10 == 0):
        print(f'{i+1}% measured.')

10% measured.
20% measured.
30% measured.
40% measured.
50% measured.
60% measured.
70% measured.
80% measured.
90% measured.
100% measured.


In [12]:
print(f'Average SIMP time: {overall_simp_time / n_samples}')
print(f'Average NN time: {overall_nn_time / n_samples}')
print(f'Ratio: {overall_simp_time / overall_nn_time}')

Average SIMP time: 3.818713645499999
Average NN time: 0.0039492509
Ratio: 966.9463253145042
